In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
import keras
from keras import backend as K
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
pd.set_option('display.max_rows', 500)


In [3]:
test_size = 0.3
val_size = 0.5
train_size =0.7

random_state = 42

In [4]:
data = pd.read_csv('../feature_sets/post_champ_sel/raw_v1.csv')
data.head()

,league,season,week,patch,format,game_in_format,blue_team_outcome,season_format,month,day,...,champ_red_ban_0_id,champ_red_ban_1_id,champ_red_ban_2_id,champ_red_ban_3_id,champ_red_ban_4_id,champ_red_top_id,champ_red_jungle_id,champ_red_mid_id,champ_red_bot_id,champ_red_supp_id
0,3,0,0,5,2,1,1,0,4,20,...,1,9,54.0,49.0,147.0,106,148,11,113,129
1,3,0,0,5,2,2,0,0,4,20,...,72,129,142.0,2.0,130.0,103,81,11,113,95
2,3,0,0,5,2,3,1,0,4,20,...,72,129,77.0,97.0,155.0,109,147,10,142,54
3,3,0,0,5,2,4,1,0,4,20,...,1,9,49.0,155.0,81.0,38,147,130,54,101
4,3,0,5,5,2,1,1,0,4,14,...,113,77,142.0,109.0,97.0,103,104,130,162,73


In [5]:
pred = data.pop('blue_team_outcome')

In [6]:
x_train, x_temp, y_train, y_temp = train_test_split(data,pred,test_size=test_size,random_state = random_state)
x_val, x_test, y_val, y_test = train_test_split(x_temp,y_temp,test_size = val_size,random_state = random_state)

In [7]:
assert len(data) == len(x_train) + len(x_test) + len(x_val)
assert len(data) == len(y_train) + len(y_val) + len(y_test)

In [8]:
num_input = len(data.columns)
num_input

95

In [9]:
model = Sequential()
model.add(Dense(1024, input_shape=(num_input,), activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16384, activation='relu'))
model.add(Dense(16384, activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [10]:
patience = 10
learning_rate = 1e-6
epochs = 200
batch_szie=128

In [11]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=0, mode='min')
mcp_save = keras.callbacks.ModelCheckpoint('mdl_wts.keras', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])

In [14]:
mcp_save

In [12]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_szie,verbose=True, validation_data=(x_val, y_val))

Epoch 1/200
5/5 [==============================] - 2s 168ms/step - loss: 1.9792 - accuracy: 0.5073 - val_loss: 0.8679 - val_accuracy: 0.4924
Epoch 2/200
5/5 [==============================] - 1s 121ms/step - loss: 1.5139 - accuracy: 0.5302 - val_loss: 0.7180 - val_accuracy: 0.6212
Epoch 3/200
5/5 [==============================] - 1s 116ms/step - loss: 1.3012 - accuracy: 0.4959 - val_loss: 1.0985 - val_accuracy: 0.5000
Epoch 4/200
5/5 [==============================] - 1s 118ms/step - loss: 1.1093 - accuracy: 0.5546 - val_loss: 0.6287 - val_accuracy: 0.6742
Epoch 5/200
5/5 [==============================] - 1s 119ms/step - loss: 0.9976 - accuracy: 0.5612 - val_loss: 0.8500 - val_accuracy: 0.5455
Epoch 6/200
5/5 [==============================] - 1s 122ms/step - loss: 1.0358 - accuracy: 0.5759 - val_loss: 0.6236 - val_accuracy: 0.6742
Epoch 7/200
5/5 [==============================] - 1s 119ms/step - loss: 0.9448 - accuracy: 0.5710 - val_loss: 0.5990 - val_accuracy: 0.6515
Epoch 8/200
5

In [13]:
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

5/5 [==============================] - 0s 11ms/step - loss: 0.6872 - accuracy: 0.6818
Accuracy: 68.18
